# Exploracion y limpieza de Datos de F1
- En este notebook cargamos los datasets, limpiamos los datos y preparamos la tabla para el análisis

## Configuracion e Importacion
- Importamos la librerias necesarias y configuramos el entorno

In [1]:
import pandas as pd
import requests
import sys
import os
import numpy as np
import ast  # Busque en python.org como solucionar la columna 'location' en api_circuits.csv, los valores de lat y lng estaban incluidos como una string, y utilice esta funcion 
            # para convertir la string en diccionario y poder extraer los valores de lat y lng para el analisis


# Visualizacion de las columnas
pd.set_option('display.max_columns', None)

# Agregacion de carpeta para importar src
sys.path.append(os.path.abspath(".."))

# Importamos función desde funciones.py
try:
    from funciones import descargar_api,limpieza_tablas
    print("Funciones  cargadas correctamente.")
except ImportError as e:
    print(f" Error al importar funciones: {e}")

# Configuracion de rutas locales
ruta_local = "../data/"
archivos = ['results.csv','races.csv','drivers.csv','constructors.csv','status.csv', 'circuits.csv']   
df = {}


# Carga de archivos locales en un diccionario de DataFrames
print('Iniciando carga de datos')

for archivo in archivos:
    # Concatenacion de los textos
    ruta_completa = ruta_local + archivo
    nombre_columna = archivo.replace('.csv','')

    try:
        df[nombre_columna] = pd.read_csv(ruta_completa)
        print(f"Tabla '{nombre_columna}' cargada ({df[nombre_columna].shape[0]} filas).")
    except FileNotFoundError:
        print(f"Error: no se encontro '{ruta_completa}'")


# Carga de archivos de la API a traves de la funcion
print("Cargando Api")

df['api_circuits'] = descargar_api()

# Comprobacion final
print("Tablas disponibles:", list(df.keys()))

limpieza_tablas(df)



Funciones  cargadas correctamente.
Iniciando carga de datos
Tabla 'results' cargada (26759 filas).
Tabla 'races' cargada (1125 filas).
Tabla 'drivers' cargada (861 filas).
Error: no se encontro '../data/constructors.csv'
Error: no se encontro '../data/status.csv'
Tabla 'circuits' cargada (77 filas).
Cargando Api
Descargando API desde https://api.jolpi.ca/ergast/f1/circuits.json?limit=100
 Guardado en: ../data/api_circuits.csv
Tablas disponibles: ['results', 'races', 'drivers', 'circuits', 'api_circuits']
Tabla results no tiene nulos
Tabla results no tiene duplicados
Tabla results tiene 74 IDs duplicados
Tabla results tiene 0 espacios en blanco en la columna number
Tabla results tiene 0 espacios en blanco en la columna position
Tabla results tiene 0 espacios en blanco en la columna positionText
Tabla results tiene 0 espacios en blanco en la columna time
Tabla results tiene 0 espacios en blanco en la columna milliseconds
Tabla results tiene 0 espacios en blanco en la columna fastestLap
T

## Validacion de Datos de la API

Aunque el dataset principal circuits.csv ya contiene la información de los circuitos, he utilizado los datos de la API api_circuits.csv  para validar la precisión de las coordenadas geográficas (latitud y longitud).

In [2]:

# Cuando ya tenia el proyecto avanzado me di cuenta que la API no estaba bien formada, por lo que tuve que hacer un pequeño script para limpiarla. 
# Busque como poder convertir location en un diccionario y asi poder darle un valor a lat y lng para comparar en la tabla de circuits que los datos geograficos de los circuitos son correctos, Encontre la funcion ast.literal_eval().
# El csv de la api me daba los datos en formato string, por lo que tuve que convertirlos a object para poder acceder a los valores de lat y lng y asi poder usarlos para determinar si lat y long en ambas tablas estaban bien.


# Transformar el texto JSON de la API en columnas reales
df_api_clean = df['api_circuits'].copy()
df_api_clean['Location_dict'] = df_api_clean['Location'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Crear columnas nuevas limpias
df_api_clean['lat_api'] = df_api_clean['Location_dict'].apply(lambda x: float(x.get('lat', 0)))
df_api_clean['lng_api'] = df_api_clean['Location_dict'].apply(lambda x: float(x.get('long', 0)))

# Eliminacion de espacios y conversion a minusculas
df['circuits']['circuitRef'] = df['circuits']['circuitRef'].astype(str).str.strip().str.lower()
df_api_clean['circuitId'] = df_api_clean['circuitId'].astype(str).str.strip().str.lower()

# Merge para pegar la info de la API al dataframe de circuits
# Usamos 'circuitRef' (circuits) y 'circuitId' (API) para la  unión
df['circuits'] = pd.merge(
    df['circuits'],
    df_api_clean[['circuitId', 'lat_api', 'lng_api']], 
    left_on='circuitRef',
    right_on='circuitId',
    how='left',
    suffixes=('', '_api')
)

print(" Datos de API integrados en la tabla de circuitos.")
df['circuits'][['circuitRef', 'lat', 'lat_api', 'lng', 'lng_api']].head()




 Datos de API integrados en la tabla de circuitos.


,circuitRef,lat,lat_api,lng,lng_api
0,albert_park,-37.84970,-37.84970,144.96800,144.96800
1,sepang,2.76083,2.76083,101.73800,101.73800
2,bahrain,26.03250,26.03250,50.51060,50.51060
3,catalunya,41.57000,41.57000,2.26111,2.26111
4,istanbul,40.95170,40.95170,29.40500,29.40500


# Identificacion de duplicados
- Antes de unir las tablas, analice la tabla results para detectar posibles inconsistencias. Identifique IDs repetidos (mismo piloto en misma carrera) para entender la naturaleza de los duplicados.

In [3]:

#  Identificacion los IDs que se repiten
subset_cols = ['raceId', 'driverId'] 
duplicados_ids = df['results'][df['results'].duplicated(subset=subset_cols, keep=False)]

#  ordenamos columnas raceID y driverID para ver por que son los duplicados
duplicados_ordenados = duplicados_ids.sort_values(by=['raceId', 'driverId'])

print(f"{len(duplicados_ordenados)} filas que generan los 74 duplicados de ID:")

duplicados_ordenados.head(10)

176 filas que generan los 74 duplicados de ID:


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
13188,13189,540,229,54,10,0,\N,F,26,0.0,0,\N,\N,\N,\N,\N,\N,81
13191,13192,540,229,57,23,0,\N,F,29,0.0,0,\N,\N,\N,\N,\N,\N,97
17362,17363,717,373,172,1,1,7,7,7,0.0,102,\N,\N,\N,\N,\N,\N,60
24298,24304,717,373,172,2,1,\N,R,12,0.0,54,\N,\N,\N,\N,\N,\N,98
17740,17741,733,465,172,48,0,\N,W,22,0.0,0,\N,\N,\N,\N,\N,\N,54
17743,17744,733,465,97,50,0,\N,W,25,0.0,0,\N,\N,\N,\N,\N,\N,54
17961,17962,742,475,102,26,20,\N,D,19,0.0,56,\N,\N,\N,\N,\N,\N,2
17963,17964,742,475,172,28,5,\N,R,21,0.0,44,\N,\N,\N,\N,\N,\N,23
18062,18063,745,418,172,22,11,\N,R,17,0.0,23,\N,\N,\N,\N,\N,\N,6
24297,24303,745,418,172,21,15,11,11,11,0.0,92,\N,\N,\N,\N,\N,\N,18


# Union de tablas
- Antes de hacer el merge, converti todos las columnas a numerico para que no hubiese ningun error.
- Despues uní las tablas results con carreras, pilotos y circuitos para obtener un solo DataFrame

In [4]:

# Conversion de que los IDs sean numericos antes del merge

df['results']['raceId'] = pd.to_numeric(df['results']['raceId'], errors='coerce')
df['races']['raceId'] = pd.to_numeric(df['races']['raceId'], errors='coerce')
df['races']['circuitId'] = pd.to_numeric(df['races']['circuitId'], errors='coerce')
df['circuits']['circuitId'] = pd.to_numeric(df['circuits']['circuitId'], errors='coerce')
df['drivers']['driverId'] = pd.to_numeric(df['drivers']['driverId'], errors='coerce')


In [5]:

#  Merge de results con carreras para obtener circuitoID
df_merge = pd.merge(
    df['results'], 
    df['races'][['raceId', 'year', 'circuitId']], 
    on='raceId', 
    how='left'
)

# Merge con drivesr  para obtener la nacionalidad del piloto
df_merge = pd.merge(
    df_merge, 
    df['drivers'][['driverId', 'nationality', 'surname']], 
    on='driverId', 
    how='left'
)

# Merge con cicuits para obtener el pais del circuito
# country aparece dentro de location

cols_circuits = ['circuitId', 'location', 'name', 'country']

df_merge = pd.merge(
    df_merge.drop(columns=['Location', 'circuitName'], errors='ignore'), 
    df['circuits'][cols_circuits], 
    on='circuitId', 
    how='left'
)

print(f" Merge completado:{df_merge.shape[0]} filas.")

 Merge completado:26759 filas.


In [6]:
df_merge

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,year,circuitId,nationality,surname,location,name,country
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1,2008,1,British,Hamilton,Melbourne,Albert Park Grand Prix Circuit,Australia
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1,2008,1,German,Heidfeld,Melbourne,Albert Park Grand Prix Circuit,Australia
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1,2008,1,German,Rosberg,Melbourne,Albert Park Grand Prix Circuit,Australia
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1,2008,1,Spanish,Alonso,Melbourne,Albert Park Grand Prix Circuit,Australia
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1,2008,1,Finnish,Kovalainen,Melbourne,Albert Park Grand Prix Circuit,Australia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26754,26760,1144,825,210,20,14,16,16,16,0.0,57,\N,\N,57,1,1:25.637,222.002,11,2024,24,Danish,Magnussen,Abu Dhabi,Yas Marina Circuit,UAE
26755,26761,1144,859,215,30,12,17,17,17,0.0,55,\N,\N,52,12,1:28.751,214.212,5,2024,24,New Zealander,Lawson,Abu Dhabi,Yas Marina Circuit,UAE
26756,26762,1144,822,15,77,9,\N,R,18,0.0,30,\N,\N,14,19,1:29.482,212.462,130,2024,24,Finnish,Bottas,Abu Dhabi,Yas Marina Circuit,UAE
26757,26763,1144,861,3,43,20,\N,R,19,0.0,26,\N,\N,5,17,1:29.411,212.631,5,2024,24,Argentinian,Colapinto,Abu Dhabi,Yas Marina Circuit,UAE


# Limpieza de los Duplicados
- Tras unir los datos, procedi a eliminar los registros duplicados detectados anteriormente. Conserve el registro con mayor puntuación (points) para cada piloto y carrera, eliminando las entradas redundantes o erróneas.

In [7]:

# Eliminacion de duplicados 

# Ordenamos por carrera y luego por puntos 
df_merge = df_merge.sort_values(by=['raceId', 'points'], ascending=[True, False])

columnas_duplicados = ['raceId', 'driverId']

# Conprobacion de filas y seleccion del primer valor

print(f"Filas origianles: {df_merge.shape[0]}")
df_merge = df_merge.drop_duplicates(subset=columnas_duplicados, keep='first')

# Conteo de filas despues de eliminar los duplicados
print(f"Filas después de eliminar duplicados: {df_merge.shape[0]}")

# Verificacion de duplicados
duplicados_restantes = df_merge.duplicated(subset=columnas_duplicados).sum()
print(f"Duplicados restantes: {duplicados_restantes}")

Filas origianles: 26759


Filas después de eliminar duplicados: 26668
Duplicados restantes: 0


# Normalizacion de columnas
- Renombre las columnas para que fuesen mas legibles 

In [8]:

# Llamada a funcion limpieza titulo de columnas
try:
    from funciones import columnas_analisis
    print("Función importada ")
except ImportError as e:
    print(f"Error {e}. ")


df_final = columnas_analisis(df_merge)
df_final.head()

Función importada 
Columnas originales: 25.
COlumnas para el analisis: 9


,año,apellido_piloto,nacionalidad_piloto,pais_circuito,posicion_salida,posicion_final,puntos,id_carrera,id_piloto
7553,2009,Button,British,Australia,1,1,10.0,1,18
7554,2009,Barrichello,Brazilian,Australia,2,2,8.0,1,22
7555,2009,Trulli,Italian,Australia,20,3,6.0,1,15
7556,2009,Glock,German,Australia,19,4,5.0,1,10
7557,2009,Alonso,Spanish,Australia,10,5,4.0,1,4


#  Columna Pais_piloto
- Cree esta función para saber si el piloto es local o no para poder realizar la segunda hipotesis para asi obtener el nombre del pais del piloto en vez de su gentilicio y poder cruzar la columna con pais_circuito

In [9]:

#Llamada a funcion pais pilito

try:
    from funciones import pais_piloto
    print("Funcion Importada")
except ImportError as e:
    print(f"Error {e}")

Funcion Importada


In [ ]:

# Aplicamos la función para traducir la nacionalidad a país
df_final['pais_piloto'] = df_final['nacionalidad_piloto'].apply(pais_piloto)



,nacionalidad_piloto,pais_piloto,pais_circuito,es_local
7553,British,UK,Australia,False
7554,Brazilian,Brazil,Australia,False
7555,Italian,Italy,Australia,False
7556,German,Germany,Australia,False
7557,Spanish,Spain,Australia,False


# Cree una ultima columna llamada es_local necesaria para la hipotesis 2

In [ ]:
# Creaacion de la columna 'es_local' comparando las dos columnas de país
df_final['es_local'] = df_final['pais_piloto'] == df_final['pais_circuito']


df_final[['nacionalidad_piloto', 'pais_piloto', 'pais_circuito', 'es_local']].head()

In [11]:
df_final

,año,apellido_piloto,nacionalidad_piloto,pais_circuito,posicion_salida,posicion_final,puntos,id_carrera,id_piloto,pais_piloto,es_local
7553,2009,Button,British,Australia,1,1,10.0,1,18,UK,False
7554,2009,Barrichello,Brazilian,Australia,2,2,8.0,1,22,Brazil,False
7555,2009,Trulli,Italian,Australia,20,3,6.0,1,15,Italy,False
7556,2009,Glock,German,Australia,19,4,5.0,1,10,Germany,False
7557,2009,Alonso,Spanish,Australia,10,5,4.0,1,4,Spain,False
...,...,...,...,...,...,...,...,...,...,...,...
26754,2024,Magnussen,Danish,UAE,14,16,0.0,1144,825,Danish,False
26755,2024,Lawson,New Zealander,UAE,12,17,0.0,1144,859,New Zealander,False
26756,2024,Bottas,Finnish,UAE,9,18,0.0,1144,822,Finnish,False
26757,2024,Colapinto,Argentinian,UAE,20,19,0.0,1144,861,Argentina,False


# Tabla para el analisis final
- Seleccion las columnas necesarias para el analisis y ademas cree una columna mas (gano_desde_pole) necesaria para la primera hipotesis

In [12]:

columnas_finales = [
    'año', 
    'apellido_piloto', 
    'pais_circuito', 
    'posicion_salida', 
    'posicion_final', 
    'puntos', 
    'id_carrera', 
    'id_piloto',
    'pais_piloto', 
    'es_local'      
]

df_analisis = df_final[columnas_finales].copy()
df_analisis['gano_desde_pole'] = (df_analisis['posicion_salida'] == 1) & (df_analisis['posicion_final'] == 1)

In [13]:
display(df_analisis)

,año,apellido_piloto,pais_circuito,posicion_salida,posicion_final,puntos,id_carrera,id_piloto,pais_piloto,es_local,gano_desde_pole
7553,2009,Button,Australia,1,1,10.0,1,18,UK,False,True
7554,2009,Barrichello,Australia,2,2,8.0,1,22,Brazil,False,False
7555,2009,Trulli,Australia,20,3,6.0,1,15,Italy,False,False
7556,2009,Glock,Australia,19,4,5.0,1,10,Germany,False,False
7557,2009,Alonso,Australia,10,5,4.0,1,4,Spain,False,False
...,...,...,...,...,...,...,...,...,...,...,...
26754,2024,Magnussen,UAE,14,16,0.0,1144,825,Danish,False,False
26755,2024,Lawson,UAE,12,17,0.0,1144,859,New Zealander,False,False
26756,2024,Bottas,UAE,9,18,0.0,1144,822,Finnish,False,False
26757,2024,Colapinto,UAE,20,19,0.0,1144,861,Argentina,False,False


# Guarde el data set final en la carpeta Data

In [14]:

# Guardar archico 

ruta_archivo = "../data/df_analisis_final.csv"
df_analisis.to_csv(ruta_archivo,index=False)
print(f"Archivo guardado en: {ruta_archivo}")

Archivo guardado en: ../data/df_analisis_final.csv
